In [1]:
import json
import os
from google.cloud import pubsub_v1
from dotenv import load_dotenv, find_dotenv
from concurrent.futures import TimeoutError
_ = load_dotenv(find_dotenv())

In [12]:
#Set up authentication using the service account key
SERVICE_ACCOUNT_KEY_PATH = "../key.json"
GCP_PROJECT_ID=os.getenv('GCP_PROJECT_ID')

# Set the environment variable for authentication
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = SERVICE_ACCOUNT_KEY_PATH
# Step 2: Define Pub/Sub parameters
PROJECT_ID = GCP_PROJECT_ID

# Define Pub/Sub parameters
SUBSCRIPTION_ID = "apache-beam-bounded-sub"
# SUBSCRIPTION_ID = "poc-apache-beam-sub"


# Initialize the Pub/Sub Subscriber Client
subscriber = pubsub_v1.SubscriberClient()
subscription_path = subscriber.subscription_path(PROJECT_ID, SUBSCRIPTION_ID)


I0000 00:00:1739148981.285430 19208415 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [13]:


# Define the callback function to process messages
def callback(message):
    print(f"\n🎯 Received message: {message.data.decode('utf-8')}")
    print(f"📌 Attributes: {message.attributes}")

    # Acknowledge the message to remove it from the queue
    message.ack()
    print("✅ Message acknowledged!\n")

In [14]:
#Start listening to messages
print(f"🚀 Listening for messages on subscription: {subscription_path} ...\n")
# Subscribe to the topic asynchronously
streaming_pull_future = subscriber.subscribe(subscription_path, callback=callback)

🚀 Listening for messages on subscription: projects/loyal-operation-213500/subscriptions/apache-beam-bounded-sub ...



In [15]:
# Keep the script running to listen for messages
try:
    streaming_pull_future.result(timeout=60)  # Listen for 60 seconds (adjust as needed)
except TimeoutError:
    streaming_pull_future.cancel()  # Stop the subscriber
    print("⏳ Listener stopped after timeout.")

⏳ Listener stopped after timeout.
